<a href="https://colab.research.google.com/github/DaiZack/GMATanalyzer/blob/master/gmat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GMAT SC analysis

## step 1 load OG data

In [238]:
# use wget to download from my github
!wget https://github.com/DaiZack/GMATanalyzer/raw/master/OG2021.txt

--2020-07-31 16:03:19--  https://github.com/DaiZack/GMATanalyzer/raw/master/OG2021.txt
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/DaiZack/GMATanalyzer/master/OG2021.txt [following]
--2020-07-31 16:03:20--  https://raw.githubusercontent.com/DaiZack/GMATanalyzer/master/OG2021.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 130907 (128K) [text/plain]
Saving to: ‘OG2021.txt’

OG2021.txt          100%[===================>] 127.84K  --.-KB/s    in 0.04s   

2020-07-31 16:03:20 (3.07 MB/s) - ‘OG2021.txt’ saved [130907/130907]



In [239]:
# read the file
with open('OG2021.txt') as ogf:
  og = '\n'+ogf.read()

In [207]:
# split the file into questions by the lines only with number
ogquestions = [o for o in re.split(r'\n\d+\n', og) if o]
numbers = re.findall(r'\n\d+\n', og)
numbers = [n.replace('\n',' ') for n in numbers]
# put the number back to the question for the convience of lookup
ogquestions = [numbers[i]+ogquestions[i] for i in range(len(ogquestions))]

In [236]:
# set up a function to find diffs in each option
def findDiff(question):
  import re
  out = [] # empty list to receive the result
  body = [sent for sent in question.split('\n') if len(sent)>10][0] # the body of question
  aopt = ''.join(re.findall(r'\nA\.? (.*)', question)) # the underline part (same as A option)
  body = re.split(aopt, body)[0] + '[' + aopt+']'+re.split(aopt, body)[-1] # add [] around the underline
  options = [q for q in question.split('\n') if re.findall(r'[ABCDE]\.? ', q)] # get 5 options

  wordlist = set(' '.join(options).split(' ')) # bag of words, all words appeared in options
  fqdict = {word:sum([word in opt for opt in options]) for word in wordlist} # how many option has each word
  out.append(body) # put question body into out
  for i,q in enumerate(options): #loop each option
    out.append(' '.join([ f'[{word}]' if word in fqdict and 
                         fqdict[word]<5 else word for word in q.split(' ') ]))
    # if the word not appear in all 5 option, mark them with []
  return '\n'.join(out) # return result as a string

In [237]:
# loop the question and write into the file
with open('ogscDiff.txt','w') as f:
  for ogq in ogquestions:
    out = findDiff(ogq)
    f.write(out+'\n\n')